In [ ]:
# Step 1: Import all necessary libraries
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

# Data preprocessing and feature engineering
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer

# Model selection and evaluation
from sklearn.model_selection import cross_val_score, StratifiedKFold, train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Models
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.ensemble import RandomForestClassifier, StackingClassifier
from sklearn.linear_model import LogisticRegression

# Imbalanced learning
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import make_pipeline
from imblearn.under_sampling import RandomUnderSampler

# Hyperparameter optimization
import optuna

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

print("All libraries imported successfully!")

In [ ]:
# Step 2: Load and explore the data
df = pd.read_csv('/kaggle/input/heart-disease-health-indicators-dataset/heart_disease_health_indicators_BRFSS2015.csv')

print("Dataset shape:", df.shape)
print("\nFirst few rows:")
print(df.head())

print("\nDataset info:")
print(df.info())

print("\nClass distribution:")
print(df['HeartDiseaseorAttack'].value_counts())
print("Percentage of positive class: {:.2f}%".format(df['HeartDiseaseorAttack'].mean() * 100))

# Check basic statistics
print("\nBasic statistics:")
print(df.describe())

In [ ]:
# Step 3: Data quality check and feature investigation
print("\n" + "="*50)
print("DATA QUALITY CHECK")
print("="*50)

# Check for missing values
print("\nMissing values:")
print(df.isnull().sum())

# Check the _RFHYPE5 column for potential leakage
print(f"\n_RFHYPE5 value counts:")
print(df['_RFHYPE5'].value_counts())

print(f"\nCross-tab of _RFHYPE5 vs HeartDiseaseorAttack:")
leakage_check = pd.crosstab(df['_RFHYPE5'], df['HeartDiseaseorAttack'], normalize='index')
print(leakage_check)

# Calculate correlation with target
correlation_with_target = df.corr()['HeartDiseaseorAttack'].sort_values(ascending=False)
print(f"\nTop 10 features correlated with target:")
print(correlation_with_target.head(10))

# Based on the high correlation, we'll drop this column to avoid potential leakage
print("\nDropping _RFHYPE5 column to avoid potential data leakage...")
df = df.drop('_RFHYPE5', axis=1)
print(f"Dataset shape after dropping _RFHYPE5: {df.shape}")

In [ ]:
# Step 4: Advanced Feature Engineering
print("\n" + "="*50)
print("FEATURE ENGINEERING")
print("="*50)

# Create interaction features and risk scores
def create_new_features(df):
    df_copy = df.copy()

    # Create interaction terms
    df_copy['BMI_Age'] = df_copy['BMI'] * df_copy['Age']
    df_copy['BP_Chol'] = df_copy['HighBP'] * df_copy['HighChol']
    df_copy['Diabetes_Smoker'] = df_copy['Diabetes'] * df_copy['Smoker']
    df_copy['Age_GenHlth'] = df_copy['Age'] * df_copy['GenHlth']

    # Create comprehensive risk scores
    df_copy['TotalRiskScore'] = (df_copy['HighBP'] + df_copy['HighChol'] +
                                df_copy['Smoker'] + df_copy['Diabetes'] +
                                df_copy['Stroke'] + df_copy['PhysActivity'])

    df_copy['LifestyleRisk'] = (df_copy['Smoker'] + df_copy['HvyAlcoholConsump'] +
                               (5 - df_copy['GenHlth']))  # Lower GenHlth = worse health

    df_copy['MedicalHistoryScore'] = (df_copy['HighBP'] + df_copy['HighChol'] +
                                     df_copy['Diabetes'] + df_copy['Stroke'])

    # Create age groups
    df_copy['AgeGroup'] = pd.cut(df_copy['Age'], bins=[0, 35, 50, 65, 80, 100],
                                labels=[1, 2, 3, 4, 5])

    # BMI categories
    df_copy['BMICategory'] = pd.cut(df_copy['BMI'], bins=[0, 18.5, 25, 30, 100],
                                   labels=[1, 2, 3, 4])

    # Fill NaN values created by cutting
    df_copy['AgeGroup'] = df_copy['AgeGroup'].cat.codes
    df_copy['BMICategory'] = df_copy['BMICategory'].cat.codes

    return df_copy

df_enhanced = create_new_features(df)
print(f"Original features: {len(df.columns)}")
print(f"After feature engineering: {len(df_enhanced.columns)}")
print("New features created: BMI_Age, BP_Chol, Diabetes_Smoker, Age_GenHlth, TotalRiskScore, LifestyleRisk, MedicalHistoryScore, AgeGroup, BMICategory")

# Show correlation of new features with target
new_features_corr = df_enhanced.corr()['HeartDiseaseorAttack'].sort_values(ascending=False)
print(f"\nTop 15 features correlated with target (including new features):")
print(new_features_corr.head(15))

In [ ]:
# Step 5: Prepare final feature set and target
X = df_enhanced.drop('HeartDiseaseorAttack', axis=1)
y = df_enhanced['HeartDiseaseorAttack']

print(f"\nFinal feature set shape: {X.shape}")
print(f"Target distribution:")
print(y.value_counts())
print(f"Positive class percentage: {y.mean():.3f}")

# List all features
print(f"\nAll features used for modeling:")
for i, col in enumerate(X.columns, 1):
    print(f"{i:2d}. {col}")

In [ ]:
# Step 6: Define advanced Optuna objective function
def objective(trial):
    # Suggest model type
    model_type = trial.suggest_categorical('model_type', ['xgb', 'lgbm', 'rf'])

    if model_type == 'xgb':
        params = {
            'n_estimators': trial.suggest_int('n_estimators', 300, 2000),
            'max_depth': trial.suggest_int('max_depth', 4, 12),
            'learning_rate': trial.suggest_float('learning_rate', 0.005, 0.3, log=True),
            'subsample': trial.suggest_float('subsample', 0.6, 1.0),
            'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0),
            'reg_alpha': trial.suggest_float('reg_alpha', 1e-4, 10.0, log=True),
            'reg_lambda': trial.suggest_float('reg_lambda', 1e-4, 10.0, log=True),
            'gamma': trial.suggest_float('gamma', 0, 5),
            'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
            'random_state': 42
        }
        model = XGBClassifier(**params, use_label_encoder=False, eval_metric='logloss')

    elif model_type == 'lgbm':
        params = {
            'n_estimators': trial.suggest_int('n_estimators', 300, 2000),
            'max_depth': trial.suggest_int('max_depth', 4, 12),
            'learning_rate': trial.suggest_float('learning_rate', 0.005, 0.3, log=True),
            'subsample': trial.suggest_float('subsample', 0.6, 1.0),
            'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0),
            'reg_alpha': trial.suggest_float('reg_alpha', 1e-4, 10.0, log=True),
            'reg_lambda': trial.suggest_float('reg_lambda', 1e-4, 10.0, log=True),
            'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
            'random_state': 42
        }
        model = LGBMClassifier(**params)

    else:  # Random Forest
        params = {
            'n_estimators': trial.suggest_int('n_estimators', 200, 1000),
            'max_depth': trial.suggest_int('max_depth', 8, 20),
            'min_samples_split': trial.suggest_int('min_samples_split', 2, 20),
            'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 10),
            'max_features': trial.suggest_float('max_features', 0.1, 1.0),
            'bootstrap': trial.suggest_categorical('bootstrap', [True, False]),
            'random_state': 42
        }
        model = RandomForestClassifier(**params)

    # Create pipeline with SMOTE
    pipeline = make_pipeline(
        SMOTE(random_state=42),
        model
    )

    # Use Stratified K-Fold Cross-Validation
    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    scores = cross_val_score(pipeline, X, y, cv=cv, scoring='accuracy', n_jobs=-1)

    return scores.mean()

print("Optuna objective function defined successfully!")

In [ ]:
# Step 7: Run hyperparameter optimization
print("\n" + "="*50)
print("HYPERPARAMETER OPTIMIZATION")
print("="*50)

print("Starting Optuna optimization with 150 trials...")
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=150, show_progress_bar=True)

print('\nBest trial:')
trial = study.best_trial
print(f'  Best Accuracy: {trial.value:.5f}')
print(f'  Best Model Type: {trial.params["model_type"]}')
print(f'  Best Params:')
for key, value in trial.params.items():
    print(f'    {key}: {value}')

# Plot optimization history
plt.figure(figsize=(10, 6))
optuna.visualization.plot_optimization_history(study)
plt.title('Optuna Optimization History')
plt.show()

In [ ]:
# Step 8: Train the best single model
print("\n" + "="*50)
print("TRAINING BEST SINGLE MODEL")
print("="*50)

best_params = trial.params.copy()
model_type = best_params.pop('model_type')

print(f"Training best {model_type.upper()} model...")

if model_type == 'xgb':
    best_model = XGBClassifier(**best_params, use_label_encoder=False, eval_metric='logloss')
elif model_type == 'lgbm':
    best_model = LGBMClassifier(**best_params)
else:
    best_model = RandomForestClassifier(**best_params)

# Create pipeline with best model
final_pipeline = make_pipeline(
    SMOTE(random_state=42),
    best_model
)

# Train on full data
final_pipeline.fit(X, y)
print("Best single model trained successfully!")

# Cross-validation score
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
cv_scores = cross_val_score(final_pipeline, X, y, cv=cv, scoring='accuracy', n_jobs=-1)
print(f"Cross-validation scores: {cv_scores}")
print(f"Mean CV Accuracy: {cv_scores.mean():.5f} (+/- {cv_scores.std() * 2:.5f})")

In [ ]:
# Step 9: Create an Ensemble Model (Stacking)
print("\n" + "="*50)
print("CREATING ENSEMBLE MODEL")
print("="*50)

# Define base models for stacking
base_models = [
    ('xgb', XGBClassifier(
        n_estimators=1000,
        max_depth=8,
        learning_rate=0.1,
        subsample=0.8,
        colsample_bytree=0.8,
        random_state=42,
        use_label_encoder=False,
        eval_metric='logloss'
    )),
    ('lgbm', LGBMClassifier(
        n_estimators=1000,
        max_depth=8,
        learning_rate=0.1,
        subsample=0.8,
        colsample_bytree=0.8,
        random_state=42
    )),
    ('rf', RandomForestClassifier(
        n_estimators=500,
        max_depth=15,
        min_samples_split=10,
        min_samples_leaf=4,
        max_features=0.7,
        random_state=42,
        n_jobs=-1
    ))
]

# Create stacking classifier
stacking_model = StackingClassifier(
    estimators=base_models,
    final_estimator=LogisticRegression(C=0.1, random_state=42, max_iter=1000),
    cv=5,
    n_jobs=-1
)

# Create pipeline for stacking
stacking_pipeline = make_pipeline(
    SMOTE(random_state=42),
    stacking_model
)

print("Training ensemble model...")
stacking_pipeline.fit(X, y)
print("Ensemble model trained successfully!")

# Cross-validation score for ensemble
cv_scores_ensemble = cross_val_score(stacking_pipeline, X, y, cv=cv, scoring='accuracy', n_jobs=-1)
print(f"Ensemble CV scores: {cv_scores_ensemble}")
print(f"Ensemble Mean CV Accuracy: {cv_scores_ensemble.mean():.5f} (+/- {cv_scores_ensemble.std() * 2:.5f})")

In [ ]:
# Step 10: Model Evaluation and Comparison
print("\n" + "="*50)
print("MODEL EVALUATION")
print("="*50)

# Split data for final evaluation
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f"Training set shape: {X_train.shape}")
print(f"Test set shape: {X_test.shape}")

# Train both models on training set for fair comparison
print("Training models on training set...")
final_pipeline.fit(X_train, y_train)
stacking_pipeline.fit(X_train, y_train)

# Make predictions
single_pred = final_pipeline.predict(X_test)
ensemble_pred = stacking_pipeline.predict(X_test)

# Calculate accuracies
single_accuracy = accuracy_score(y_test, single_pred)
ensemble_accuracy = accuracy_score(y_test, ensemble_pred)

print(f"\nSingle Model Test Accuracy: {single_accuracy:.5f}")
print(f"Ensemble Model Test Accuracy: {ensemble_accuracy:.5f}")

# Choose the best model
if ensemble_accuracy >= single_accuracy:
    best_final_model = stacking_pipeline
    best_accuracy = ensemble_accuracy
    model_name = "Ensemble Stacking Model"
else:
    best_final_model = final_pipeline
    best_accuracy = single_accuracy
    model_name = "Single Best Model"

print(f"\nSelected {model_name} with accuracy: {best_accuracy:.5f}")

# Detailed classification report
print(f"\nDetailed Classification Report for {model_name}:")
if model_name == "Ensemble Stacking Model":
    y_pred = ensemble_pred
else:
    y_pred = single_pred

print(classification_report(y_test, y_pred))

# Confusion matrix
cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.title(f'Confusion Matrix - {model_name}')
plt.ylabel('Actual')
plt.xlabel('Predicted')
plt.show()

In [ ]:
# Step 11: Feature Importance Analysis
print("\n" + "="*50)
print("FEATURE IMPORTANCE ANALYSIS")
print("="*50)

def get_feature_importance(model, feature_names):
    """Extract feature importance from different model types"""
    if hasattr(model, 'feature_importances_'):
        # For single tree-based models
        return model.feature_importances_
    elif hasattr(model, 'estimators_'):
        # For stacking classifier - use the first base model
        return model.estimators_[0].feature_importances_
    elif hasattr(model, 'coef_'):
        # For linear models
        return np.abs(model.coef_[0])
    else:
        return None

# Get the actual model from the pipeline
if model_name == "Ensemble Stacking Model":
    actual_model = best_final_model.steps[-1][1]
else:
    actual_model = best_final_model.steps[-1][1]

feature_importances = get_feature_importance(actual_model, X.columns)

if feature_importances is not None:
    feature_importance_df = pd.DataFrame({
        'feature': X.columns,
        'importance': feature_importances
    }).sort_values('importance', ascending=False)

    print("\nTop 15 Most Important Features:")
    print(feature_importance_df.head(15))

    # Plot feature importance
    plt.figure(figsize=(12, 8))
    sns.barplot(data=feature_importance_df.head(15), x='importance', y='feature')
    plt.title('Top 15 Feature Importances')
    plt.tight_layout()
    plt.show()
else:
    print("Feature importance not available for this model type.")

In [ ]:
# Step 12: Final Model Training and Saving
print("\n" + "="*50)
print("FINAL MODEL TRAINING AND SAVING")
print("="*50)

# Retrain the best model on the full dataset
print("Training final model on full dataset...")
best_final_model.fit(X, y)

# Final cross-validation
cv_scores_final = cross_val_score(best_final_model, X, y, cv=5, scoring='accuracy', n_jobs=-1)
print(f"Final Cross-Validation Scores: {cv_scores_final}")
print(f"Final Mean CV Accuracy: {cv_scores_final.mean():.5f} (+/- {cv_scores_final.std() * 2:.5f})")

# Save the model
import joblib

model_filename = 'heart_disease_best_model.pkl'
feature_filename = 'feature_names.pkl'

joblib.dump(best_final_model, model_filename)
joblib.dump(list(X.columns), feature_filename)

print(f"\nModel saved as: {model_filename}")
print(f"Feature names saved as: {feature_filename}")

# Final summary
print("\n" + "="*50)
print("FINAL SUMMARY")
print("="*50)
print(f"Best Model: {model_name}")
print(f"Final CV Accuracy: {cv_scores_final.mean():.5f}")
print(f"Number of Features: {X.shape[1]}")
print(f"Dataset Size: {X.shape[0]} samples")
print(f"Positive Class Ratio: {y.mean():.3f}")

In [ ]:
# Step 13: Reality Check
print("\n" + "="*60)
print("REALITY CHECK - IMPORTANT NOTES")
print("="*60)

print("""
⚠️ ACHIEVING 99.5% ACCURACY - REALITY CHECK:

1. MEDICAL DATA CHALLENGES:
   - Real-world medical data rarely achieves >99% accuracy
   - High accuracy might indicate data leakage or overfitting
   - Focus on robust cross-validation scores, not just test accuracy

2. VERIFICATION STEPS:
   - Ensure no target leakage in features
   - Check cross-validation consistency
   - Validate on completely independent dataset if possible

3. ALTERNATIVE METRICS:
   - For medical data, RECALL (sensitivity) is often more important than accuracy
   - Precision and F1-score provide better picture of model performance
   - Consider ROC-AUC for binary classification

4. NEXT STEPS FOR IMPROVEMENT:
   - Try neural networks with careful regularization
   - Experiment with different feature combinations
   - Consider feature selection to remove noise
   - Try more sophisticated ensemble methods

Current performance is excellent if CV accuracy > 90%!
""")